In [1]:
import pandas as pd
import numpy as np
#import dask.dataframe as dd
#import dask.distributed
import dask.dataframe as dd
import dask.distributed

#Nov15
ddfile = 'Z:/DiDi/gps_20161115.txt'
di = dd.read_csv(ddfile, names = ["DriverID", "OrderID", "TimeStamp", "Longitude","Latitude"])
di.head()

,DriverID,OrderID,TimeStamp,Longitude,Latitude
0,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194294,104.07323,30.69225
1,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194296,104.07323,30.69225
2,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194300,104.07323,30.69225
3,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194303,104.07323,30.69225
4,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194306,104.07323,30.69225


In [2]:
#filter out the data by buffer (30 m on both sides)
%time
trips = di[(di.Longitude>=104.047) & (di.Longitude<=104.083)]
#zone=zone.compute()
trips.head()

Wall time: 0 ns


,DriverID,OrderID,TimeStamp,Longitude,Latitude
0,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194294,104.07323,30.69225
1,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194296,104.07323,30.69225
2,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194300,104.07323,30.69225
3,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194303,104.07323,30.69225
4,cbd8237f1b71595dbfc963c439e8ee00,ba70af5bd30a9026e756c82314291192,1479194306,104.07323,30.69225


In [3]:
%time
trips = trips[(trips.Latitude<=30.695) & (trips.Latitude>=30.694)]
trips=trips.compute()
trips.head()

Wall time: 0 ns


,DriverID,OrderID,TimeStamp,Longitude,Latitude
441,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202200,104.04755,30.69414
442,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202206,104.04809,30.69419
443,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202209,104.04824,30.69420
444,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202212,104.04837,30.69421
445,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202215,104.04845,30.69422


In [4]:
trips_2 = trips
order_count = trips_2.groupby(by = 'OrderID', as_index = False).count()
bad_ids = order_count[order_count['DriverID'] == 1].OrderID

In [5]:
Good_trips = trips_2[~trips_2['OrderID'].isin(bad_ids)]

In [6]:
import datetime
print(
    datetime.datetime.fromtimestamp(int("1478750813")).strftime('%Y-%m-%d %H:%M:%S')
)

2016-11-09 23:06:53


In [7]:
Good_trips['time_date'] = Good_trips['TimeStamp'].apply(lambda x : datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
Good_trips['date_time'] = pd.to_datetime(Good_trips['time_date'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
Good_trips['HOUR-Min'] = Good_trips['TimeStamp'].apply(lambda x : datetime.datetime.fromtimestamp(int(x)).strftime('%H:%M'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df = Good_trips

In [11]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km


In [12]:
len(df)

1539248

In [13]:
df['Longitude_shifted'] = df.Longitude.shift()
df['Latitude_shifted'] = df.Latitude.shift()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
import time
s0 = time.time()

df['Dist'] = df.apply(lambda row: haversine_np(row.Longitude_shifted, row.Latitude_shifted, row.Longitude, row.Latitude), axis=1)

e0 = time.time()
print("Time taken by all the trips is {}.".format((e0-s0)))

Time taken by all the trips is 241.57999992370605.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
df = df[~df.Dist.isnull()]

In [16]:
#df.head()
#df1 = df[['OrderID', 'TimeStamp', 'Longitude', 'Latitude', 'time_date', 'date_time', 'HOUR-Min', 'Dist', 'Dist_mile' ]]

In [17]:
df['time_shifted'] = df.TimeStamp.shift()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
df.head()

,DriverID,OrderID,TimeStamp,Longitude,Latitude,time_date,date_time,HOUR-Min,Longitude_shifted,Latitude_shifted,Dist,time_shifted
442,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202206,104.04809,30.69419,2016-11-15 04:30:06,2016-11-15 04:30:06,04:30,104.04755,30.69414,0.051899,NaN
443,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202209,104.04824,30.69420,2016-11-15 04:30:09,2016-11-15 04:30:09,04:30,104.04809,30.69419,0.014377,1.479202e+09
444,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202212,104.04837,30.69421,2016-11-15 04:30:12,2016-11-15 04:30:12,04:30,104.04824,30.69420,0.012472,1.479202e+09
445,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202215,104.04845,30.69422,2016-11-15 04:30:15,2016-11-15 04:30:15,04:30,104.04837,30.69421,0.007725,1.479202e+09
446,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202218,104.04850,30.69422,2016-11-15 04:30:18,2016-11-15 04:30:18,04:30,104.04845,30.69422,0.004778,1.479202e+09


In [19]:
s0 = time.time()

df['time_interval'] = df['TimeStamp']- df['time_shifted']
e0 = time.time()
print("Time taken by all the trips is {}.".format((e0-s0)))

Time taken by all the trips is 0.05499982833862305.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
df.head()

,DriverID,OrderID,TimeStamp,Longitude,Latitude,time_date,date_time,HOUR-Min,Longitude_shifted,Latitude_shifted,Dist,time_shifted,time_interval
442,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202206,104.04809,30.69419,2016-11-15 04:30:06,2016-11-15 04:30:06,04:30,104.04755,30.69414,0.051899,NaN,NaN
443,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202209,104.04824,30.69420,2016-11-15 04:30:09,2016-11-15 04:30:09,04:30,104.04809,30.69419,0.014377,1.479202e+09,3.0
444,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202212,104.04837,30.69421,2016-11-15 04:30:12,2016-11-15 04:30:12,04:30,104.04824,30.69420,0.012472,1.479202e+09,3.0
445,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202215,104.04845,30.69422,2016-11-15 04:30:15,2016-11-15 04:30:15,04:30,104.04837,30.69421,0.007725,1.479202e+09,3.0
446,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202218,104.04850,30.69422,2016-11-15 04:30:18,2016-11-15 04:30:18,04:30,104.04845,30.69422,0.004778,1.479202e+09,3.0


In [21]:
df = df[~df.time_interval.isnull()]

In [22]:
df['Dist_mile'] = df['Dist']* 0.621371

In [23]:
df['speed'] =(df['Dist_mile']/ df['time_interval'])* 3600
df.head()

,DriverID,OrderID,TimeStamp,Longitude,Latitude,time_date,date_time,HOUR-Min,Longitude_shifted,Latitude_shifted,Dist,time_shifted,time_interval,Dist_mile,speed
443,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202209,104.04824,30.69420,2016-11-15 04:30:09,2016-11-15 04:30:09,04:30,104.04809,30.69419,0.014377,1.479202e+09,3.0,0.008933,10.719807
444,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202212,104.04837,30.69421,2016-11-15 04:30:12,2016-11-15 04:30:12,04:30,104.04824,30.69420,0.012472,1.479202e+09,3.0,0.007750,9.299690
445,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202215,104.04845,30.69422,2016-11-15 04:30:15,2016-11-15 04:30:15,04:30,104.04837,30.69421,0.007725,1.479202e+09,3.0,0.004800,5.760034
446,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202218,104.04850,30.69422,2016-11-15 04:30:18,2016-11-15 04:30:18,04:30,104.04845,30.69422,0.004778,1.479202e+09,3.0,0.002969,3.562577
447,20a72cb1d5402ec420bcb05d9b5022c3,4fe8135b0154058a6f1d3c20d0af3ac2,1479202221,104.04853,30.69422,2016-11-15 04:30:21,2016-11-15 04:30:21,04:30,104.04850,30.69422,0.002867,1.479202e+09,3.0,0.001781,2.137546


In [24]:
df = df[df['speed'] >= 0]

In [25]:
df = df[df['speed'] <= 300]

In [26]:
df_Morn = df[(df['HOUR-Min'] <= '09:00') & (df['HOUR-Min'] >= '06:00')]
df_Morn.head()

,DriverID,OrderID,TimeStamp,Longitude,Latitude,time_date,date_time,HOUR-Min,Longitude_shifted,Latitude_shifted,Dist,time_shifted,time_interval,Dist_mile,speed
5643,588987caf0e37d8bb98abe7907f0b35f,ade0e572ccffb901a490b24897ccca5c,1479211296,104.08130,30.69494,2016-11-15 07:01:36,2016-11-15 07:01:36,07:01,104.07988,30.69496,0.135708,1.479177e+09,34320.0,0.084325,0.008845
5644,588987caf0e37d8bb98abe7907f0b35f,ade0e572ccffb901a490b24897ccca5c,1479211299,104.08127,30.69447,2016-11-15 07:01:39,2016-11-15 07:01:39,07:01,104.08130,30.69494,0.052307,1.479211e+09,3.0,0.032502,39.002774
16029,4eabb7578609ce3ccaa168cae912cb99,7d9bd1dd15de8aaba1973b422f553c1e,1479210336,104.08251,30.69468,2016-11-15 06:45:36,2016-11-15 06:45:36,06:45,104.04708,30.69432,3.385808,1.479199e+09,11695.0,2.103843,0.647613
16030,4eabb7578609ce3ccaa168cae912cb99,7d9bd1dd15de8aaba1973b422f553c1e,1479210339,104.08209,30.69468,2016-11-15 06:45:39,2016-11-15 06:45:39,06:45,104.08251,30.69468,0.040134,1.479210e+09,3.0,0.024938,29.925507
16031,4eabb7578609ce3ccaa168cae912cb99,7d9bd1dd15de8aaba1973b422f553c1e,1479210342,104.08176,30.69468,2016-11-15 06:45:42,2016-11-15 06:45:42,06:45,104.08209,30.69468,0.031534,1.479210e+09,3.0,0.019594,23.512898


In [27]:
df_Mid = df[(df['HOUR-Min'] <= '15:00') & (df['HOUR-Min'] >= '09:00')]
df_Mid.head()

,DriverID,OrderID,TimeStamp,Longitude,Latitude,time_date,date_time,HOUR-Min,Longitude_shifted,Latitude_shifted,Dist,time_shifted,time_interval,Dist_mile,speed
21164,95bf69cf71af6d728c7f094c1c06cd75,567724dd28b453a61da2fa6fada796ce,1479218649,104.08135,30.69423,2016-11-15 09:04:09,2016-11-15 09:04:09,09:04,104.04732,30.69451,3.251945,1.479180e+09,38157.0,2.020664,0.190644
21165,95bf69cf71af6d728c7f094c1c06cd75,567724dd28b453a61da2fa6fada796ce,1479218652,104.08137,30.69468,2016-11-15 09:04:12,2016-11-15 09:04:12,09:04,104.08135,30.69423,0.050043,1.479219e+09,3.0,0.031095,37.314179
26960,7ab5278016916b7db5e4a5f9bcde5eb4,334868584ec4fb1c6ce4e928acf47146,1479220585,104.05737,30.69490,2016-11-15 09:36:25,2016-11-15 09:36:25,09:36,104.04951,30.69487,0.751079,1.479217e+09,3516.0,0.466699,0.477849
26961,7ab5278016916b7db5e4a5f9bcde5eb4,334868584ec4fb1c6ce4e928acf47146,1479220588,104.05736,30.69462,2016-11-15 09:36:28,2016-11-15 09:36:28,09:36,104.05737,30.69490,0.031130,1.479221e+09,3.0,0.019343,23.211712
26962,7ab5278016916b7db5e4a5f9bcde5eb4,334868584ec4fb1c6ce4e928acf47146,1479220591,104.05733,30.69432,2016-11-15 09:36:31,2016-11-15 09:36:31,09:36,104.05736,30.69462,0.033461,1.479221e+09,3.0,0.020791,24.949707


In [28]:
df_Even = df[(df['HOUR-Min'] <= '18:00') & (df['HOUR-Min'] >= '15:00')]
df_night = df[(df['HOUR-Min'] <= '22:00') & (df['HOUR-Min'] >= '18:00')]

In [29]:
df_agg_nov_Morn = df_Morn.groupby([df_Morn['date_time'].dt.floor('5Min')]).agg(['mean'])

In [30]:
df_agg_nov_Morn['TT_min'] = ((2.14)/(df_agg_nov_Morn['speed']))*60

In [31]:
df_agg_nov_Morn.head()

,TimeStamp,Longitude,Latitude,Longitude_shifted,Latitude_shifted,Dist,time_shifted,time_interval,Dist_mile,speed,TT_min
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,
date_time,,,,,,,,,,,
2016-11-15 06:00:00,1.479208e+09,104.065525,30.694585,104.065567,30.694584,0.109587,1.479207e+09,991.080776,0.068094,19.582137,6.556996
2016-11-15 06:05:00,1.479208e+09,104.064017,30.694577,104.064024,30.694578,0.095562,1.479207e+09,705.455507,0.059379,18.825560,6.820514
2016-11-15 06:10:00,1.479208e+09,104.065382,30.694576,104.065447,30.694575,0.085292,1.479208e+09,662.850189,0.052998,15.758155,8.148162
2016-11-15 06:15:00,1.479209e+09,104.064777,30.694579,104.064782,30.694578,0.078001,1.479208e+09,498.532503,0.048467,17.014630,7.546447
2016-11-15 06:20:00,1.479209e+09,104.066421,30.694600,104.066388,30.694600,0.079103,1.479209e+09,457.343868,0.049153,16.790488,7.647187


In [32]:
df_agg_nov_Morn.to_csv('Nov 15 morning am.csv') 

In [33]:
df_agg_nov_mid = df_Mid.groupby([df_Mid['date_time'].dt.floor('5Min')]).agg(['mean'])


In [34]:
df_agg_nov_mid['TT_min'] = ((2.14)/(df_agg_nov_mid['speed']))*60

In [35]:
df_agg_nov_mid.to_csv('Nov 15 mid pm.csv') 

In [36]:
df_agg_nov_even = df_Even.groupby([df_Even['date_time'].dt.floor('5Min')]).agg(['mean'])

In [37]:
df_agg_nov_even['TT_min'] = ((2.14)/(df_agg_nov_even['speed']))*60

In [38]:
df_agg_nov_even.to_csv('Nov 15 evening pm.csv') 

In [39]:
df_agg_nov_night = df_night.groupby([df_night['date_time'].dt.floor('5Min')]).agg(['mean'])
df_agg_nov_night['TT_min'] = ((2.14)/(df_agg_nov_night['speed']))*60

In [40]:
df_agg_nov_night.to_csv('Nov 15 night till 10 pm.csv') 

In [41]:
df_agg_nov_night.head()

,TimeStamp,Longitude,Latitude,Longitude_shifted,Latitude_shifted,Dist,time_shifted,time_interval,Dist_mile,speed,TT_min
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,
date_time,,,,,,,,,,,
2016-11-14 18:00:00,1.479165e+09,104.063133,30.694561,104.063098,30.694561,0.031905,1.479165e+09,37.056395,0.019825,17.629580,7.283214
2016-11-14 18:05:00,1.479165e+09,104.066217,30.694595,104.066105,30.694596,0.028371,1.479165e+09,41.612429,0.017629,15.897802,8.076588
2016-11-14 18:10:00,1.479165e+09,104.067543,30.694586,104.067489,30.694585,0.034375,1.479165e+09,50.088034,0.021359,18.279565,7.024237
2016-11-14 18:15:00,1.479165e+09,104.065587,30.694608,104.065602,30.694610,0.030112,1.479165e+09,21.627488,0.018711,18.336084,7.002586
2016-11-14 18:20:00,1.479166e+09,104.065525,30.694589,104.065471,30.694591,0.034411,1.479166e+09,39.225086,0.021382,20.949965,6.128889


In [ ]:
df['Dist_mile'] = df['Dist']* 0.621371

In [ ]:
df.head()

In [ ]:
df['speed'] = df['Dist_mile']* 1200
df.head()

In [ ]:
#morning peak:
df_Morn = df[(df['HOUR-Min'] <= '09:00') & (df['HOUR-Min'] >= '06:00')]
#df_Mid = df[(df['HOUR-Min'] <= '15:00') & (df['HOUR-Min'] >= '09:00')]
#df_Even = df[(df['HOUR-Min'] <= '18:00') & (df['HOUR-Min'] >= '15:00')]
#df_night = df[(df['HOUR-Min'] <= '12:00') & (df['HOUR-Min'] >= '18:00')]

In [ ]:
df_agg_nov2['TT_min'] = ((2.14)/(df_agg_nov2['speed']))*60

In [ ]:
df_agg_nov2.head()

In [ ]:
df_Morn = df[(df['HOUR-Min'] <= '09:00') & (df['HOUR-Min'] >= '06:00')]

In [ ]:
df_Morn.to_csv('Nov 2 morning am')